In [1]:
!pip install sacrebleu konlpy torch tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 21.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-many

In [2]:
import os

# 데이터 준비
def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.readlines()
    return [line.strip() for line in content]

# 파일 경로 설정 및 데이터 읽기
directory_path = '/content/drive/MyDrive/구름/Week4/archive'
file_names = ['je.train', 'ko.train', 'je.dev', 'ko.dev', 'je.test', 'ko.test']
variable_names = ['je_train', 'ko_train', 'je_dev', 'ko_dev', 'je_test', 'ko_test']

for file_name, variable_name in zip(file_names, variable_names):
    file_path = os.path.join(directory_path, file_name)
    content = read_text_file(file_path)
    globals()[variable_name] = content

# 학습 데이터를 일부만 사용
je_train = je_train[0:10000]
ko_train = ko_train[0:10000]
ko_test = ko_test[0:1000]
je_test = je_test[0:1000]

# 문장에 태그 추가
tagged_ko_train = ["<2je> " + sentence for sentence in ko_train]
tagged_je_train = ["<2ko> " + sentence for sentence in je_train]

tagged_ko_test = ["<2je> " + sentence for sentence in ko_test]
tagged_je_test = ["<2ko> " + sentence for sentence in je_test]

# 학습 및 테스트 데이터 설정
train_src_texts = tagged_ko_train + tagged_je_train
train_tgt_texts = je_train + ko_train

test_src_texts = tagged_ko_test + tagged_je_test
test_tgt_texts = je_test + ko_test

In [3]:
import torch
import torch.nn as nn
import json
from konlpy.tag import Okt
import sacrebleu

# KonlpyTokenizer 클래스 정의
class KonlpyTokenizer:
    def __init__(self):
        self.okt = Okt()
        self.word2idx = {}
        self.idx2word = {}
        self.vocab_size = 0

    def load(self, file_path):
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            self.word2idx = data['word2idx']
            self.idx2word = {int(k): v for k, v in data['idx2word'].items()}
            self.vocab_size = len(self.word2idx)

    def encode(self, sentence):
        return [self.word2idx.get(word, self.word2idx['<unk>']) for word in self.okt.morphs(sentence)]

    def decode(self, tokens):
        return ' '.join([self.idx2word[token] for token in tokens if token != 0])

# Seq2Seq 모델 정의
class Seq2SeqModel(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model=512, num_layers=2, dropout=0.1):
        super(Seq2SeqModel, self).__init__()
        self.encoder = nn.LSTM(d_model, d_model, num_layers, dropout=dropout, batch_first=True)
        self.decoder = nn.LSTM(d_model, d_model, num_layers, dropout=dropout, batch_first=True)
        self.src_tok_emb = nn.Embedding(src_vocab_size, d_model)
        self.tgt_tok_emb = nn.Embedding(tgt_vocab_size, d_model)
        self.fc_out = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, tgt):
        src_emb = self.dropout(self.src_tok_emb(src))
        tgt_emb = self.dropout(self.tgt_tok_emb(tgt))
        _, (hidden, cell) = self.encoder(src_emb)
        output, _ = self.decoder(tgt_emb, (hidden, cell))
        return self.fc_out(output)

# 모델 및 토크나이저 로드
model_save_path_ko_to_je = '/content/drive/MyDrive/transformer_translation_model_ko_to_je.pth'
model_save_path_je_to_ko = '/content/drive/MyDrive/transformer_translation_model_je_to_ko.pth'
tokenizer_save_path = '/content/drive/MyDrive/tokenizer.json'

tokenizer = KonlpyTokenizer()
tokenizer.load(tokenizer_save_path)

src_vocab_size = tokenizer.vocab_size
tgt_vocab_size = tokenizer.vocab_size
model_ko_to_je = Seq2SeqModel(src_vocab_size, tgt_vocab_size)
model_je_to_ko = Seq2SeqModel(tgt_vocab_size, src_vocab_size)

# 모델을 CPU 장치로 매핑하여 로드
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_ko_to_je.load_state_dict(torch.load(model_save_path_ko_to_je, map_location=device))
model_je_to_ko.load_state_dict(torch.load(model_save_path_je_to_ko, map_location=device))
model_ko_to_je.to(device)
model_je_to_ko.to(device)

# 번역 함수 정의
def translate(model, tokenizer, sentence, max_length=128):
    model.eval()
    src_ids = tokenizer.encode(sentence)[:max_length]
    src_ids = torch.tensor(src_ids).unsqueeze(0).to(device).long()  # LongTensor로 변환
    tgt_ids = [tokenizer.word2idx['<pad>']] * max_length
    tgt_ids = torch.tensor(tgt_ids).unsqueeze(0).to(device).long()  # LongTensor로 변환

    with torch.no_grad():
        src_emb = model.src_tok_emb(src_ids)  # 임베딩 적용
        _, (hidden, cell) = model.encoder(src_emb)
        for i in range(1, max_length):
            tgt_emb = model.tgt_tok_emb(tgt_ids[:, :i])  # 임베딩 적용
            output, (hidden, cell) = model.decoder(tgt_emb, (hidden, cell))
            pred_token = output.argmax(2)[:, -1].item()
            tgt_ids[0, i] = pred_token
            if pred_token == tokenizer.word2idx['<pad>']:
                break

    return tokenizer.decode(tgt_ids[0].cpu().numpy().astype(int))  # int로 변환하여 디코드


# BLEU 점수 계산
def calculate_bleu(model, tokenizer, src_texts, tgt_texts):
    references = [[tgt] for tgt in tgt_texts]
    hypotheses = [translate(model, tokenizer, src) for src in src_texts]
    bleu = sacrebleu.corpus_bleu(hypotheses, references)
    return bleu.score

In [4]:
ko_train[0:10]

['판관했던 거 ?',
 '우리 그냥 맨손에 맨손에 김도 매고 , 김도 베고 . 그렇게 했었어 , 옛날은 .',
 '예 .',
 '응 , 그러니까 .',
 '운영해 오다가 .',
 '에 , 밭벼 , 아 요는 뭐이든지 복잡하게 나면 위 다퉈서 되지를 않아 .',
 '아 , 오리목 말하는 거 . 오리목이지 .',
 '보통 지붕 위로 올리는데 어떤 데는 올레 길에 하는 데도 있는 모양이야 .',
 '여름에는 목장에 올리고 .',
 '예 .']

In [5]:
# 예제 번역 수행
ko_to_je = [
    "예"
]

print("ko_to_je 번역 결과:")
for sentence in ko_to_je:
    translation = translate(model_ko_to_je, tokenizer, sentence)
    print(f"Original: {sentence}")
    print(f"Translated: {translation}")

ko_to_je 번역 결과:
Original: 예
Translated: 매어서 꼰저 꼰저 웃터 웃터 웃터 웃터 웃터 가네 가네 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어 찧어


In [6]:
je_train[0:10]

['판관헤난 거 ?',
 '우리 그냥 맨손에 맨손에 검질도 메고 , 검질도 비고 . 경헤난 , 옛날은 .',
 '에 .',
 '응 , 거니까 .',
 '운영헤 오다가 .',
 '에 , 산듸 , 아 요는 뭐이던지 복잡허게 나면은 우 ᄃᆞ탕은에 뒈들 아녀 .',
 '아 , 오리목 말허는 거 . 오릿목입주게 .',
 '보통 지붕 우로 올리는데 어떤 디는 올레 질에 허는 디도 잇는 모양이라 .',
 '여름에는 목장에 올리고 .',
 '예 .']

In [7]:
je_to_ko = [
    "오릿목입주게 ."
]

print("je_to_ko 번역 결과:")
for sentence in je_to_ko:
    translation = translate(model_je_to_ko, tokenizer, sentence)
    print(f"Original: {sentence}")
    print(f"Translated: {translation}")

je_to_ko 번역 결과:
Original: 오릿목입주게 .
Translated: 똥 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사 라사


In [8]:
# ko_to_je 번역 성능 평가
bleu_score_ko_to_je = calculate_bleu(model_ko_to_je, tokenizer, test_src_texts, test_tgt_texts)
print(f"BLEU Score (ko_to_je): {bleu_score_ko_to_je}")

BLEU Score (ko_to_je): 0.27732513448329266


In [9]:
# je_to_ko 번역 성능 평가
bleu_score_je_to_ko = calculate_bleu(model_je_to_ko, tokenizer, test_tgt_texts, test_src_texts)
print(f"BLEU Score (je_to_ko): {bleu_score_je_to_ko}")

BLEU Score (je_to_ko): 0.32209783553187415
